In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import geopandas as gpd

In [4]:
df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


@interact
def show_articles_more_than(column='pop_est', x=5000000):
    return df.loc[df[column] > x]

interactive(children=(Text(value='pop_est', description='column'), IntSlider(value=5000000, description='x', m…

In [12]:
# Interact with specification of arguments
@interact
def show_articles_more_than(column=['pop_est', 'gdp_md_est'], 
                            x=(10, 1000000, 10)):
    return df.loc[df[column] > x]

interactive(children=(Dropdown(description='column', options=('pop_est', 'gdp_md_est'), value='pop_est'), IntS…

Now we get a dropdown for the column (with the options in the list) and an integer slider limited to a range (the format is (start, stop, step) ). Read through the documentation for the full details of how function parameters are mapped to widgets.
We can use this same @interact decorator to quickly turn any ordinary function into an interactive widget. For example, we may have a lot of images in a directory we want to quickly look through:

In [13]:
import os
from IPython.display import Image
@interact
def show_images(file=os.listdir('images/')):
    display(Image(fdir+file))

FileNotFoundError: [Errno 2] No such file or directory: 'images/'

# Widget for Plots

In [5]:
import cufflinks as cf

@interact_manual
def scatter_plot(x=list(df.select_dtypes('number').columns), 
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()), 
                 colorscale=list(cf.colors._scales_names.keys())):
    
    df.iplot(kind='scatter', x=x, y=y, mode='markers', 
             xTitle=x.title(), yTitle=y.title(), 
             title=f'{y.title()} vs {x.title()}',
            theme=theme, colorscale=colorscale)

interactive(children=(Dropdown(description='x', options=('pop_est', 'gdp_md_est'), value='pop_est'), Dropdown(…

# Expanding the Capabilities of Interactive Controls


In [6]:
# Create interactive version of function with DatePickers
interact(stats_for_article_published_between,
        start_date=widgets.DatePicker(value=pd.to_datetime('2018-01-01')),
        end_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

NameError: name 'stats_for_article_published_between' is not defined

If we want to make the options for one widget dependent on the value of another, we use the observe function. Here, we alter the image browser function to choose both the directory and image. The list of images displayed is updated based on the directory we select.

In [8]:
import os
# Create widgets
directory = widgets.Dropdown(options=['images', 'nature', 'assorted'])
images = widgets.Dropdown(options=os.listdir(directory.value))

# Updates the image options based on directory value
def update_images(*args):
    images.options = os.listdir(directory.value)

# Tie the image options to directory value
directory.observe(update_images, 'value')

# Show the images
def show_images(fdir, file):
    display(Image(f'{fdir}/{file}'))

_ = interact(show_images, fdir=directory, file=images)

FileNotFoundError: [Errno 2] No such file or directory: 'images'

# Reusable Widgets


In [9]:
def show_stats_by_tag(tag):
    return(df.groupby(f'<tag>{tag}').describe()[['views', 'reads']])
    
stats = interact(show_stats_by_tag,
                tag=widgets.Dropdown(options=['Towards Data Science', 'Education', 
                                              'Machine Learning', 'Python', 'Data Science']))

interactive(children=(Dropdown(description='tag', options=('Towards Data Science', 'Education', 'Machine Learn…